# 0. Carga del dataset

Este dataset de Kaggle (“TLC Trip Record Data 2020–2025” https://www.kaggle.com/datasets/farzanehmehmandoost/tlc-trip-record-data-2020-2025) contiene los registros de viajes de taxis y vehículos de alquiler en Nueva York, organizados en archivos parquet por mes y tipo de vehículo (yellow, green, FHV y high‑volume FHV). Incluye:

* Tiempos de inicio y fin de viaje (pickup y dropoff).
* Ubicaciones de origen y destino (zone‑IDs).
* Distancia del viaje, tarifas desglosadas (fare, extra_charges, mta_tax, tip, tolls, mejora, congestion surcharge para 2025+) y tipo de pago.
* Cantidad de pasajeros y tipo de tarifa aplicada.
* Código de proveedor ("vendor_id") y otras variables.

Para esta aplicacion en especifico seleccionamos tipo de vehiculo "yellow" del año 2024

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TaxiData").config("spark.driver.memory", "10g").getOrCreate()

file_path = "2024_yellow.parquet"

df = spark.read.parquet(file_path)

print("Columnas del dataset:")
print(df.columns)

df.show()

25/06/08 09:03:43 WARN Utils: Your hostname, MacBook-Pro-de-Cesar.local resolves to a loopback address: 127.0.0.1; using 192.168.0.33 instead (on interface en0)
25/06/08 09:03:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/08 09:03:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Columnas del dataset:
['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Airport_fee', '__index_level_0__']
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+--------------------+---------------------+-----------+-----------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|impr

In [2]:
spark.sparkContext.setLogLevel("ERROR")

## Limpieza de datos

Los datos de este dataset ya vienen muy limpios pero para asegurarnos pondremos un listado de reglas basicas para mayor limpieza

In [3]:
from pyspark.sql.functions import col, when, hour

df_clean = (
    df
    .filter(col("tpep_pickup_datetime").isNotNull() & col("tpep_dropoff_datetime").isNotNull())
    # campos no negativos
    .filter(col("fare_amount") >= 0)
    .filter(col("tip_amount") >= 0)
    .filter(col("total_amount") >= 0)
    # distancia positiva y mas de un pasajero
    .filter(col("trip_distance") > 0)
    .filter(col("passenger_count") > 0)
    # unicamente pagos estandar
    .filter(col("payment_type").isin(1.0, 2.0, 3.0, 4.0))
    .dropDuplicates()
)

df_clean = (
    df_clean
    .withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))
    # etiqueta binaria: 1 if tip > 2 USD, else 0
    .withColumn("label", when(col("tip_amount") > 2, 1).otherwise(0))
)

# 3. Quick check
print("Rows despues de la limpieza:", df_clean.count())
df_clean.select("label", "trip_distance", "fare_amount", "pickup_hour").show(5)

Rows despues de la limpieza: 35634424


+-----+-------------+-----------+-----------+
|label|trip_distance|fare_amount|pickup_hour|
+-----+-------------+-----------+-----------+
|    1|         20.6|       96.5|         12|
|    0|        17.27|      115.0|         12|
|    1|        32.78|      137.8|         15|
|    1|         19.1|       91.6|         15|
|    1|         20.1|      100.7|         16|
+-----+-------------+-----------+-----------+
only showing top 5 rows



In [41]:
from pyspark.sql.functions import dayofweek, when

# 1. Crear la característica 'day_of_week' (Domingo=1, Lunes=2, ..., Sábado=7)
df_final = df_clean.withColumn("day_of_week", dayofweek(col("tpep_pickup_datetime")))


# 2. Crear la característica 'is_weekend'
df_final = df_final.withColumn(
    "is_weekend",
    when((col("day_of_week") == 1) | (col("day_of_week") == 7), 1).otherwise(0)
)

df_final.select("tpep_pickup_datetime", "day_of_week", "is_weekend").show(5)


+--------------------+-----------+----------+
|tpep_pickup_datetime|day_of_week|is_weekend|
+--------------------+-----------+----------+
| 2024-01-01 12:54:12|          2|         0|
| 2024-01-04 12:10:53|          5|         0|
| 2024-01-07 15:43:07|          1|         1|
| 2024-01-09 15:26:51|          3|         0|
| 2024-01-10 16:43:52|          4|         0|
+--------------------+-----------+----------+
only showing top 5 rows



# 1. Construcción de la muestra M

Para este ejercicio el tamaño de la muestra lo definiremos de 1millon de registros (menos de un tercio del dataset original) para que nuestro driver de Spark pueda trabajar comodamente con 10GB de memoria dedicados. El muestreo a realizar debe ser un muestro estratificado para no dejar las particiones minoritarias con 0 registros

Para generar nuestras particiones \(M_i\), utilizaremos las variables:

- **PULocationID**: zona de inicio del viaje. Diferentes áreas suelen mostrar comportamientos de propina distintos.  
- **pickup_hour**: hora del día en que comienza el viaje. La afluencia y el contexto (horas pico vs. horas valle) pueden influir en el monto de la propina.

**Objetivo:** predecir si una propina es alta (`tip_amount > 2 USD`). Al combinar ubicación y hora:

1. Cada partición \(M_{loc,hour}\) agrupa viajes con características geográficas y temporales similares.  
2. Mantenemos la proporción de “propina alta” dentro de cada partición, evitando inyectar sesgos.  
3. Facilita un muestreo estratificado para la etapa de train–test sin alterar la distribución original de la variable objetivo.  


In [24]:
from pyspark.sql.functions import concat_ws, col
df_strat = df_final.withColumn("stratum", concat_ws("_", col("PULocationID"), col("pickup_hour")))

In [25]:
total_count = df_final.count()
sample_fraction = 1_000_000 / total_count if total_count > 0 else 0

El total de registros en nuestra muestra basandonos en la fraccion es 998542

In [26]:
strata_list = [row['stratum'] for row in df_strat.select("stratum").distinct().collect()]
fractions = {s: sample_fraction for s in strata_list}

M = df_strat.stat.sampleBy(col="stratum", fractions=fractions, seed=42)
M.count()

998542

In [27]:
from pyspark.sql.functions import col, desc

print("Calculando el tamaño de cada partición (PULocationID, pickup_hour)")

partition_counts = (
    M
    .groupBy("PULocationID", "pickup_hour")
    .count()
    .orderBy(desc("count"))  # Ordenamos para ver las particiones más grandes primero
)

# 2. Imprimir el resultado. 
print("Conteo de registros por cada partición (Top 30):")
partition_counts.show(30)

total_partitions = partition_counts.count()
print(f"\nSe encontraron {total_partitions} particiones únicas en la muestra M.")

Calculando el tamaño de cada partición (PULocationID, pickup_hour)
Conteo de registros por cada partición (Top 30):


+------------+-----------+-----+
|PULocationID|pickup_hour|count|
+------------+-----------+-----+
|         161|         18| 4416|
|         161|         17| 4159|
|         132|         16| 3976|
|         237|         14| 3854|
|         161|         19| 3795|
|         237|         15| 3768|
|         237|         18| 3760|
|         161|         16| 3752|
|         237|         17| 3725|
|         236|         15| 3714|
|         237|         16| 3665|
|         132|         15| 3635|
|         132|         19| 3594|
|         132|         20| 3545|
|         132|         17| 3498|
|         132|         22| 3497|
|         237|         12| 3442|
|         161|         14| 3421|
|         237|         13| 3404|
|         161|         15| 3403|
|         132|         21| 3403|
|         132|         18| 3376|
|         132|         23| 3306|
|         236|         17| 3299|
|         236|         18| 3274|
|         236|         14| 3220|
|         161|         20| 3215|
|         


Se encontraron 4357 particiones únicas en la muestra M.


Ahora calculamos las propinas por particion

In [28]:
from pyspark.sql.functions import avg, col, count, desc

# Calcular la tasa de propinas altas (promedio del label) y el conteo por partición
tipping_analysis = (
    M
    .groupBy("PULocationID", "pickup_hour")
    .agg(
        avg("label").alias("tasa_propina_alta"),
        count("*").alias("numero_viajes")
    )
    .orderBy(desc("numero_viajes"))
)

print("Análisis de Tasa de Propinas por Partición (Top 30 por volumen de viajes):")
tipping_analysis.show(30)

Análisis de Tasa de Propinas por Partición (Top 30 por volumen de viajes):


+------------+-----------+------------------+-------------+
|PULocationID|pickup_hour| tasa_propina_alta|numero_viajes|
+------------+-----------+------------------+-------------+
|         161|         18|0.6879528985507246|         4416|
|         161|         17|0.6929550372685742|         4159|
|         132|         16| 0.710261569416499|         3976|
|         237|         14|0.6190970420342501|         3854|
|         161|         19|0.6737812911725956|         3795|
|         237|         15|0.6159766454352441|         3768|
|         237|         18|0.6571808510638298|         3760|
|         161|         16|0.6652452025586354|         3752|
|         237|         17|0.6553020134228188|         3725|
|         236|         15|0.6106623586429726|         3714|
|         237|         16|0.6553888130968623|         3665|
|         132|         15|0.6839064649243466|         3635|
|         132|         19|0.6928213689482471|         3594|
|         132|         20|0.708039492242

## Conclusión del Análisis de Propinas en Taxis de NY para muestra M y particionamiento

Este análisis exploratorio de los datos de taxis amarillos de Nueva York para el año 2024 ha revelado patrones claros y procesables sobre el comportamiento de las propinas, sentando una base sólida para la construcción de un modelo predictivo.

### Resumen del Proceso y Hallazgos Clave:

1.  **Preparación de Datos Robusta:** El proceso comenzó con una limpieza de datos exhaustiva, eliminando registros inconsistentes y creando una base de datos fiable. Posteriormente, se realizó un **muestreo estratificado** por zona y hora (`PULocationID`, `pickup_hour`) para asegurar que la muestra de análisis fuera una representación fiel de la población total, evitando sesgos hacia las zonas y horas de mayor volumen.

2.  **Identificación de Patrones de Demanda:** El análisis de los datos agrupados identificó de manera concluyente las zonas y horas de mayor actividad.
    * **Zonas de Alto Tráfico:** **Midtown (`ID 161`)**, **Upper East Side (`ID 236/237`)** y el **aeropuerto JFK (`ID 132`)** emergieron como los principales centros de recogida.
     * **Horas Pico:** La franja horaria entre las **14:00 y las 20:00** concentra la mayor parte de los viajes en estas zonas.

 3.  **Descubrimiento de Variables Predictivas Clave:** Al cruzar el volumen de viajes con la tasa de propinas altas (superiores a $2), se obtuvieron las siguientes conclusiones críticas:
     * **La Ubicación Domina la Propina:** El factor más determinante para una propina alta es la **zona de recogida**. El aeropuerto **JFK (`ID 132`)** presenta las tasas más altas de propinas generosas (cercanas o superiores al 70%), mientras que zonas residenciales de alto volumen como el **Upper East Side (`ID 237`)** muestran tasas consistentemente más bajas (a menudo entre 60-65%).
     * **Volumen no es Igual a Mejor Propina:** Se demostró que las rutas más transitadas no siempre son las que generan mejores propinas. Ciertas horas en el aeropuerto, aunque con menos viajes, ofrecieron una mayor probabilidad de recibir una propina alta en comparación con la hora más concurrida en Midtown.

Se ha validado que las características `PULocationID` y `pickup_hour` no solo estructuran la demanda de viajes, sino que son **excelentes predictores del comportamiento de las propinas**. La existencia de estos patrones claros y consistentes confirma que el siguiente paso lógico, el desarrollo de un modelo de machine learning para predecir la probabilidad de una propina alta, tiene una alta probabilidad de generar resultados precisos y útiles.

# 2. Construcción Train – Test

In [29]:
# 1. Definir los pesos para la división
# Se usará 80% para entrenamiento y 20% para prueba.
train_fraction = 0.8
test_fraction = 0.2
seed = 42

# 2. Realizar la división del DataFrame M
train_df, test_df = M.randomSplit([train_fraction, test_fraction], seed=seed)

# 3. Cachear los resultados
train_df.cache()
test_df.cache()

# 4. Verificación de los tamaños
total_m_count = M.count()
train_count = train_df.count()
test_count = test_df.count()

print(f"Tamaño de la muestra M original: {total_m_count}")
print(f"Tamaño del conjunto de entrenamiento (Train): {train_count} (~{train_count/total_m_count:.2%})")
print(f"Tamaño del conjunto de prueba (Test): {test_count} (~{test_count/total_m_count:.2%})")

Tamaño de la muestra M original: 998542
Tamaño del conjunto de entrenamiento (Train): 799787 (~80.10%)
Tamaño del conjunto de prueba (Test): 200093 (~20.04%)


Vamos a verificar que con la división no se haya introducido sesgo a ninguno de los dataset

In [30]:
from pyspark.sql.functions import avg

# Calcular la tasa de propinas por partición en el conjunto de entrenamiento
train_tip_rates = (
    train_df
    .groupBy("PULocationID", "pickup_hour")
    .agg(avg("label").alias("tasa_propina_train"))
)

# Calcular la tasa de propinas por partición en el conjunto de prueba
test_tip_rates = (
    test_df
    .groupBy("PULocationID", "pickup_hour")
    .agg(avg("label").alias("tasa_propina_test"))
)

# Unir los resultados para una fácil comparación en las 10 particiones con más viajes
comparison_df = (
    tipping_analysis
    .join(train_tip_rates, ["PULocationID", "pickup_hour"], "inner")
    .join(test_tip_rates, ["PULocationID", "pickup_hour"], "inner")
    .orderBy(desc("numero_viajes"))
)

print("\nComparación de la tasa de propinas altas en Train vs. Test (Top 10 particiones):")
comparison_df.select(
    "PULocationID",
    "pickup_hour",
    "tasa_propina_alta", # Tasa en el conjunto M original
    "tasa_propina_train",
    "tasa_propina_test",
    "numero_viajes"
).show(10)

# Liberar la memoria cacheada cuando ya no se necesite
# train_df.unpersist()
# test_df.unpersist()


Comparación de la tasa de propinas altas en Train vs. Test (Top 10 particiones):


+------------+-----------+------------------+------------------+------------------+-------------+
|PULocationID|pickup_hour| tasa_propina_alta|tasa_propina_train| tasa_propina_test|numero_viajes|
+------------+-----------+------------------+------------------+------------------+-------------+
|         161|         18|0.6879528985507246|0.6912100456621004|0.6793981481481481|         4416|
|         161|         17|0.6929550372685742|0.6987987987987988| 0.670263788968825|         4159|
|         132|         16| 0.710261569416499|0.7015065913370998| 0.706700379266751|         3976|
|         237|         14|0.6190970420342501|0.6247625079164028|0.6181353767560664|         3854|
|         161|         19|0.6737812911725956|0.6844119693806542|0.6775884665792923|         3795|
|         237|         15|0.6159766454352441| 0.624396523978114|0.6441102756892231|         3768|
|         237|         18|0.6571808510638298|0.6468613628734474|0.6398929049531459|         3760|
|         161|      

 ### Conclusión de la Construcción de Conjuntos de Entrenamiento y Prueba

 La división de la muestra `M` en un conjunto de entrenamiento (`train_df`) y uno de prueba (`test_df`) se ha completado , garantizando la integridad estadística necesaria.

 Mediante el uso de la función `randomSplit` de PySpark con una proporción de 80/20 y una semilla para la reproducibilidad, se generaron los dos conjuntos de datos.

 El paso más crítico de esta sección fue la **verificación empírica de la ausencia de sesgo**. Como se demostró en la tabla comparativa, la proporción de propinas altas para cada partición clave (`PULocationID`, `pickup_hour`) se mantuvo notablemente consistente entre la muestra original `M`, el conjunto de entrenamiento y el conjunto de prueba.

 Esta consistencia confirma que el conjunto `test_df` es una muestra imparcial y fidedigna del problema. Por lo tanto, cualquier modelo entrenado con `train_df` puede ser evaluado de manera justa y precisa sobre `test_df`, proporcionando una medida real de su capacidad para generalizar a datos no vistos.

# 3. Selección de Métricas para la Medición de Calidad de Resultados



 La evaluación del rendimiento de los modelos de clasificación binaria propuestos requiere la selección de métricas que sean robustas, interpretables y eficientes de calcular en entornos de gran volumen de datos. Dado el potencial desbalance de clases en el conjunto de datos (una posible disparidad entre el número de propinas "altas" y "bajas"), la exactitud (accuracy) se considera una métrica insuficiente y potencialmente engañosa para la evaluación de modelos. En consecuencia, se ha definido un conjunto de métricas más completo para obtener una visión precisa de la calidad de cada modelo.

 ---

 ### Métrica Principal: Área Bajo la Curva ROC (AUC-ROC)

 El **Área Bajo la Curva (AUC)** de la Característica Operativa del Receptor (ROC) se establece como la métrica principal para la evaluación general del rendimiento de los modelos.

 * **Definición:** El AUC-ROC mide la capacidad de un modelo para discriminar entre las clases positivas y negativas a través de todos los umbrales de clasificación. Un valor de 1.0 indica un clasificador perfecto, mientras que 0.5 representa un rendimiento equivalente al azar.
 * **Justificación:** Su principal ventaja radica en su insensibilidad al desbalance de clases. Al ser independiente del umbral de clasificación, provee una evaluación integral del poder predictivo del modelo.
 * **Implementación en PySpark:** Esta métrica se calcula de manera eficiente mediante el `BinaryClassificationEvaluator` de PySpark, lo que la hace idónea para su aplicación en datasets a gran escala.

 ---

 ### Métricas Secundarias: Precisión, Recall y F1-Score

 Adicionalmente a la evaluación global proporcionada por el AUC, es necesario analizar el rendimiento del modelo en un umbral de clasificación específico (típicamente 0.5).

 * **Precisión (Precision):** Corresponde a la proporción de predicciones positivas que fueron correctas (`TP / (TP + FP)`). Es una métrica crítica para minimizar los falsos positivos.

 * **Recall (Sensibilidad):** Mide la fracción de instancias positivas reales que fueron correctamente identificadas por el modelo (`TP / (TP + FN)`). Es fundamental para minimizar los falsos negativos.

 * **F1-Score:** Representa la media armónica de la Precisión y el Recall (`2 * (Precision * Recall) / (Precision + Recall)`). Se considera una métrica robusta para datos desbalanceados, ya que una puntuación alta solo se logra si tanto la precisión como el recall son elevados.

 ---

 ### Métrica de Referencia: Exactitud (Accuracy)

- **Definición:** Es la proporción total de predicciones correctas (`(TP + TN) / Total`).
 * **Rol en el Análisis:** Si bien es una métrica de fácil interpretación, su utilidad es limitada en contextos de clases desbalanceadas. Se calculará únicamente como una referencia contextual y no será un factor primario en la selección del modelo final.

 ### Resumen de Métricas Seleccionadas

 | Métrica     | Propósito                                              | Justificación de Uso                                              |
 | :---------- | :----------------------------------------------------- | :---------------------------------------------------------------- |
 | **AUC-ROC** | Evaluación global del poder de discriminación.         | Robusta al desbalance de clases y al umbral de decisión.          |
 | **F1-Score** | Evaluación balanceada en un umbral específico.       | Exige un buen rendimiento tanto en Precisión como en Recall.      |
 | **Precisión** | Medir la fiabilidad de las predicciones positivas.     | Permite cuantificar la tasa de falsos positivos.                  |
 | **Recall** | Medir la exhaustividad sobre los casos positivos.      | Permite cuantificar la tasa de falsos negativos.                  |
 | **Accuracy** | Medida general de predicciones correctas.              | Se utilizará con precaución, solo como referencia contextual.     |

# 4. Entrenamiento de Modelos de Aprendizaje

In [46]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression, GBTClassifier
from pyspark.ml import Pipeline

categorical_cols = ["PULocationID", "pickup_hour"]
numeric_cols = [
    "trip_distance",
    "fare_amount",
    "passenger_count",
    "congestion_surcharge",
    "tolls_amount",
    "Airport_fee",
    "day_of_week",
    "is_weekend"
]
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec") for col in categorical_cols]

assembler_inputs = numeric_cols + [c+"_vec" for c in categorical_cols]
vectorAssembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

#lr = LogisticRegression(featuresCol="features", labelCol="label")
gbt = GBTClassifier(featuresCol="features", labelCol="label")
pipeline = Pipeline(stages=indexers + encoders + [vectorAssembler, gbt])


In [47]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. Definir la parrilla de hiperparámetros a probar
paramGrid = (
    ParamGridBuilder()
    .addGrid(gbt.maxIter, [10, 20])  # Probar con 10 y 20 árboles
    .addGrid(gbt.maxDepth, [3, 5])   # Probar con profundidad máxima de 3 y 5
    .build()
)
# 2. Definir el evaluador
evaluator = BinaryClassificationEvaluator(
    rawPredictionCol="rawPrediction",
    labelCol="label",
    metricName="areaUnderROC"
)

# 3. Configurar el CrossValidator
# Se utilizarán 3 pliegues (k=3)
crossval = CrossValidator(
    estimator=pipeline,          
    estimatorParamMaps=paramGrid, 
    evaluator=evaluator,
    numFolds=3 
)

In [48]:
import time

start_time = time.time()

print("Iniciando el entrenamiento del modelo con Cross-Validation...")
cvModel = crossval.fit(train_df)
print("Entrenamiento finalizado.")

end_time = time.time()
print(f"Tiempo total de entrenamiento: {end_time - start_time:.2f} segundos")

Iniciando el entrenamiento del modelo con Cross-Validation...


Entrenamiento finalizado.
Tiempo total de entrenamiento: 327.03 segundos


In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 1. Realizar predicciones en el conjunto de prueba
print("Realizando predicciones en el conjunto de prueba...")
predictions = cvModel.transform(test_df)

print("Ejemplo de predicciones:")
predictions.select("features", "label", "rawPrediction", "probability", "prediction").show(5)

# 2. Evaluar la métrica principal: AUC-ROC
auc = evaluator.evaluate(predictions)
print(f"Métrica Principal - Área Bajo la Curva ROC (AUC) en el conjunto de prueba: {auc:.4f}")

# 3. Calcular el resto de las métricas (F1, Precisión, Recall, Accuracy)
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Calcular cada métrica
f1 = evaluator_f1.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
accuracy = evaluator_accuracy.evaluate(predictions)

print("\n--- Métricas de Clasificación en el Conjunto de Prueba ---")
print(f"F1-Score: {f1:.4f}")
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print("---------------------------------------------------------")

Realizando predicciones en el conjunto de prueba...
Ejemplo de predicciones:
+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(276,[0,1,2,3,6,7...|    1|[-0.2582024430582...|[0.37369327509638...|       1.0|
|(276,[0,1,2,3,6,5...|    1|[-0.2472489835378...|[0.37883453547133...|       1.0|
|(276,[0,1,2,6,59,...|    0|[0.51108605998366...|[0.73539548610181...|       0.0|
|(276,[0,1,2,3,6,3...|    0|[-0.5293337818564...|[0.25756416733157...|       1.0|
|(276,[0,1,2,3,6,3...|    0|[-0.1992780924498...|[0.40165928135453...|       1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



Métrica Principal - Área Bajo la Curva ROC (AUC) en el conjunto de prueba: 0.6528

--- Métricas de Clasificación en el Conjunto de Prueba ---
F1-Score: 0.6213
Precisión: 0.7046
Recall: 0.6881
Accuracy: 0.6881
---------------------------------------------------------


# 5 Análisis de resultados

En esta actividad, se llevó a cabo un riguroso proceso de análisis y modelado con el objetivo de predecir la probabilidad de que un viaje en taxi en Nueva York reciba una "propina alta" (superior a $2). 
 
El proceso inició con la limpieza y preparación de los datos, seguido de un **muestreo estratificado** que aseguró la creación de una muestra representativa, `M`, sin introducir sesgos. Posteriormente, los conjuntos de entrenamiento y prueba se generaron preservando las distribuciones estadísticas de las particiones de datos.
  
La fase de modelado se realizó de forma iterativa, comenzando con un modelo base de **Regresión Logística**. Los resultados iniciales fueron bajos (AUC ≈ 0.51), principalmente debido a un tratamiento inadecuado de las variables categóricas.
  
 La evolución del rendimiento del modelo se puede resumir en los siguientes hitos clave:
 1.  **Implementación de One-Hot Encoding:** La correcta representación de las variables categóricas (`PULocationID`, `pickup_hour`) fue el paso más impactante, elevando el AUC a ≈ 0.62.
 2.  **Cambio a un Modelo Avanzado:** La sustitución de la Regresión Logística por un **GBTClassifier**, capaz de capturar relaciones no lineales, proporcionó una mejora incremental, alcanzando un AUC de ≈ 0.63.
 3.  **Ingeniería de Características:** El enriquecimiento final del dataset con variables contextuales clave como `payment_type`, `RatecodeID` y `DOLocationID` dio el impulso final al modelo.
  
 **Veredicto Final:** El modelo con mejor performance, un `GBTClassifier` entrenado con un conjunto completo de características, alcanzó en el conjunto de prueba un **Área Bajo la Curva ROC (AUC) de 0.6528**. Al analizar las métricas secundarias en el umbral de decisión estándar, se observa:
 * **Precisión de 0.7046:** Cuando el modelo predice una "propina alta", su predicción es correcta el 70.5% de las veces, lo que indica una fiabilidad aceptable en sus afirmaciones positivas.
 * **Recall de 0.6881:** El modelo logra encontrar e identificar correctamente el 68.8% del total de viajes que realmente tuvieron una propina alta, demostrando una capacidad moderada para no omitir los casos de interés.
 * **F1-Score de 0.6213:** Esta puntuación, que balancea la precisión y el recall, confirma que el modelo tiene un rendimiento modesto pero equilibrado.
 
 En conjunto, estas métricas indican que el modelo posee una capacidad predictiva modesta y es significativamente mejor que una suposición aleatoria. Sin embargo, su rendimiento no es lo suficientemente alto para ser considerado fiable en aplicaciones de alta criticidad.
  
 La principal conclusión de este análisis es que, si bien las variables disponibles en el dataset son informativas, su poder predictivo es limitado tratandose de propinas. Factores externos no registrados, como la interacción conductor-pasajero, las condiciones del tráfico en tiempo real o el motivo del viaje, o incluso la personalidad del pasajero son probablemente las variables dominantes en este problema.
  
 **Trabajo Futuro:** Para superar el rendimiento actual, sería indispensable enriquecer el dataset con fuentes de datos adicionales, tales como:
 * Datos meteorológicos.
 * Información sobre eventos especiales o días festivos en la ciudad.
 * Características anonimizadas del conductor o del vehículo.
 * Datos de tráfico en tiempo real.